In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import utils

In [24]:
class Residual(nn.Module):
    """
        残差模块
    """
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = lambda x: x
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        """
            主路与短路保持同样的shape
        """
        # 主路
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        # 短路
        X = self.conv3(X)
        # 主路与短路复合
        Y += X
        return F.relu(Y)

In [25]:
X = torch.rand(4, 3, 6, 6)

blk = Residual(3, 3)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

In [26]:
blk = Residual(3, 5, use_1x1conv=True, strides=2)
Y = blk(X)
Y.shape

torch.Size([4, 5, 3, 3])

In [27]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), 
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

In [28]:
def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [29]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [30]:
# ResNet-18
# b1(1) + b2(4) + b3(4) + b4(4) + b5(4) + Linear(1) = 18层
net = nn.Sequential(
    b1, b2, b3, b4, b5,
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(), 
    nn.Linear(512, 10)
)

In [31]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    if isinstance(layer, nn.Sequential):
        for inner_layer in layer:
            X = inner_layer(X)
            print(inner_layer.__class__.__name__, 'output shape:\t', X.shape)
    else:
        X = layer(X)
        print(layer.__class__.__name__, 'output shape:\t', X.shape)

Conv2d output shape:	 torch.Size([1, 64, 112, 112])
BatchNorm2d output shape:	 torch.Size([1, 64, 112, 112])
ReLU output shape:	 torch.Size([1, 64, 112, 112])
MaxPool2d output shape:	 torch.Size([1, 64, 56, 56])
Residual output shape:	 torch.Size([1, 64, 56, 56])
Residual output shape:	 torch.Size([1, 64, 56, 56])
Residual output shape:	 torch.Size([1, 128, 28, 28])
Residual output shape:	 torch.Size([1, 128, 28, 28])
Residual output shape:	 torch.Size([1, 256, 14, 14])
Residual output shape:	 torch.Size([1, 256, 14, 14])
Residual output shape:	 torch.Size([1, 512, 7, 7])
Residual output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [32]:
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size, resize=96)
utils.train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-06-01 01:57:29]
epoch: 1/10, loss 0.493, train acc 0.827, test acc 0.881
epoch: 2/10, loss 0.254, train acc 0.907, test acc 0.844
epoch: 3/10, loss 0.192, train acc 0.929, test acc 0.880
epoch: 4/10, loss 0.145, train acc 0.948, test acc 0.896
epoch: 5/10, loss 0.114, train acc 0.960, test acc 0.850
epoch: 6/10, loss 0.081, train acc 0.972, test acc 0.889
epoch: 7/10, loss 0.058, train acc 0.980, test acc 0.898
epoch: 8/10, loss 0.039, train acc 0.988, test acc 0.800
epoch: 9/10, loss 0.026, train acc 0.992, test acc 0.905
epoch: 10/10, loss 0.010, train acc 0.998, test acc 0.910
*** 964.9 examples/sec on cuda:0 - [0:00:10:21], all: [0:00:12:23] ***
